In [17]:
import pandas as pd
from flask import Flask, request, jsonify
import threading
from flask_cors import CORS
import requests

In [18]:
df2019 = pd.read_csv("conteos_mensuales2019.csv")
df2019["estado"] = df2019["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2019 = df2019.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2019[["Año", "Mes"]] = df2019["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2019 = df2019.drop(columns=["year_month"])

In [19]:
df2020 = pd.read_csv("conteos_mensuales2020.csv")
df2020["estado"] = df2020["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2020 = df2020.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2020[["Año", "Mes"]] = df2020["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2020 = df2020.drop(columns=["year_month"])

In [20]:
df2021 = pd.read_csv("conteos_mensuales_2021.csv")
df2021["estado"] = df2021["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2021 = df2021.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2021[["Año", "Mes"]] = df2021["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2021 = df2021.drop(columns=["year_month"])

In [21]:
df2022 = pd.read_csv("conteos_mensuales_2022.csv")
df2022["estado"] = df2022["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2022 = df2022.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2022[["Año", "Mes"]] = df2022["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2022 = df2022.drop(columns=["year_month"])

In [22]:
app = Flask(__name__)
CORS(app)

In [23]:
datasets = {"2019": df2019, "2020": df2020, "2021": df2021, "2022": df2022}

In [24]:
def fetch_holidays(api_key, country, year):
    url = "https://calendarific.com/api/v2/holidays"
    params = {"api_key": api_key, "country": country, "year": year}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()  # Obtener el contenido JSON de la respuesta
        holidays = data.get("response", {}).get("holidays", [])
        return holidays
    else:
        print(f"Error: {response.status_code}")
        return []


# Llave API, país y años de ejemplo
api_key = "5Sqyan7wZsyg4nMiQzHKqMSxWq7oGThI"
country = "MX"
years = [2019, 2020, 2021, 2022]

# Obtener los datos de las efemérides para cada año
holidays_by_year = {year: fetch_holidays(api_key, country, year) for year in years}

In [25]:
# Extraer la información relevante
def extract_holiday_info(holidays):
    extracted_info = []
    for holiday in holidays:
        year = holiday['date']['datetime']['year']
        month = holiday['date']['datetime']['month']
        day = holiday['date']['datetime']['day']
        name = holiday['name']
        locations = holiday['locations']
        extracted_info.append({
            'year': year,
            'month': month,
            'day': day,
            'name': name,
            'locations': locations
        })
    return extracted_info

# Extraer la información de las efemérides para cada año
extracted_holidays_by_year = {year: extract_holiday_info(holidays_by_year[year]) for year in holidays_by_year}

In [26]:
# Filtrar las efemérides por año y mes
def filter_holidays_by_year_and_month(extracted_holidays_by_year, year, month):
    if year not in extracted_holidays_by_year:
        return []

    return [
        holiday
        for holiday in extracted_holidays_by_year[year]
        if holiday["month"] == month
    ]

In [27]:
@app.route("/get_data_by_year", methods=["POST"])
def get_data_by_year():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    año = request_data.get("año")

    if año not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[año]
    result = df[df["estado"] == estado]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result = result.to_dict(orient="records")
    return jsonify(result), 200

In [28]:
@app.route("/get_data", methods=["POST"])
def get_data():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    mes = int(request_data.get("mes"))
    año = int(request_data.get("año"))

    if str(año) not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[str(año)]
    result = df[(df["estado"] == estado) & (df["Mes"] == f"{mes:02}")]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result_data = result.to_dict(orient="records")

    # Obtener las festividades para el año y mes solicitados
    holidays = filter_holidays_by_year_and_month(extracted_holidays_by_year, año, mes)

    response = {"data": result_data, "holidays": holidays}

    return jsonify(response), 200

In [29]:
@app.route('/get_holidays_by_year', methods=['POST'])
def get_holidays_by_year():
    request_data = request.get_json()
    año = int(request_data.get('año'))

    if año not in extracted_holidays_by_year:
        return jsonify({"error": "Año no válido"}), 400

    holidays = extracted_holidays_by_year[año]

    if not holidays:
        return jsonify({"error": "No se encontraron efemérides para el año proporcionado"}), 404

    return jsonify(holidays), 200

In [30]:
def run_app():
    app.run(debug=False, use_reloader=False)


# Ejecutar el servidor Flask en un hilo separado
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [32]:
import requests
import json

url = "http://127.0.0.1:5000/get_data_by_year"
headers = {"Content-Type": "application/json"}
data = {"estado": "Ciudad de México", "año": "2019"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

[{'Año': '2019', 'Mes': '01', 'conteo': 341, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '02', 'conteo': 397, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '03', 'conteo': 544, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '04', 'conteo': 400, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '05', 'conteo': 405, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '06', 'conteo': 501, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '07', 'conteo': 618, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '08', 'conteo': 892, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '09', 'conteo': 718, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '10', 'conteo': 771, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '11', 'conteo': 579, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '12', 'conteo': 761, 'estado': 'CIUDADDEMÉXICO'}]


In [33]:
import requests
import json

url = "http://127.0.0.1:5000/get_data_by_year"
headers = {"Content-Type": "application/json"}
data = {"estado": "Ciudad de México", "año": "2019"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

[{'Año': '2019', 'Mes': '01', 'conteo': 341, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '02', 'conteo': 397, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '03', 'conteo': 544, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '04', 'conteo': 400, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '05', 'conteo': 405, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '06', 'conteo': 501, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '07', 'conteo': 618, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '08', 'conteo': 892, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '09', 'conteo': 718, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '10', 'conteo': 771, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '11', 'conteo': 579, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '12', 'conteo': 761, 'estado': 'CIUDADDEMÉXICO'}]


In [34]:
import requests
import json

url = "http://127.0.0.1:5000/get_data"
headers = {"Content-Type": "application/json"}
data = {"estado": "Aguascalientes", "año": "2019", "mes": "4"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

{'data': [{'Año': '2019', 'Mes': '04', 'conteo': 23328, 'estado': 'AGUASCALIENTES'}], 'holidays': [{'day': 14, 'locations': 'All', 'month': 4, 'name': 'Palm Sunday', 'year': 2019}, {'day': 18, 'locations': 'All', 'month': 4, 'name': 'Maundy Thursday', 'year': 2019}, {'day': 19, 'locations': 'All', 'month': 4, 'name': 'Good Friday', 'year': 2019}, {'day': 20, 'locations': 'All', 'month': 4, 'name': 'Holy Saturday', 'year': 2019}, {'day': 21, 'locations': 'All', 'month': 4, 'name': 'Easter Sunday', 'year': 2019}, {'day': 30, 'locations': 'All', 'month': 4, 'name': "Children's Day", 'year': 2019}]}
